## Importer av paket och data från REMbox

In [ ]:
import pandas as pd 
import hvplot.pandas #noqa #plotpaket
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from rembox_integration_tools import REMboxDataQuery
from rembox_integration_tools.rembox_analysis import StudyColumn, SeriesColumn
from pathlib import Path

# om plotly önskas så skrivs följande hvplot.extension("plotly")
hvplot.extension("bokeh")

CLIENT_ID_ENV_VAR = "REMBOX_INT_CLIENT_ID"
CLIENT_PWD_ENV_VAR = "REMBOX_INT_CLIENT_PWD"
TOKEN_URI = "https://autoqa.vll.se/dpqaauth/connect/token" #Var finns access token
API_URI = "https://rembox.vll.se/api" #Var finns API:t
ORIGIN_URI = "https://rembox.vll.se" #Vilken URL

rembox = REMboxDataQuery(
    client_id_environment_variable=CLIENT_ID_ENV_VAR,
    client_secret_environment_variable=CLIENT_PWD_ENV_VAR,
    token_uri=TOKEN_URI,
    api_uri=API_URI,
    origin_uri=ORIGIN_URI,
    verify_ssl_cert=False
)

valid_study_columns = StudyColumn()
valid_series_columns = SeriesColumn()

In [ ]:
rembox.reset_filter_options()
def get_data_from_fluoro(rembox: REMboxDataQuery) -> tuple[pd.DataFrame, pd.DataFrame]:

    
    rembox.filter_options.set_inclusive_tags(
        machine_types=["XASTAT"],     # CT-CT, Fluoroscopic-XASTAT, Mobile C-arm-XAMOB, Conventional-DX, Mammography-MG, Intraoral-IO, Panoramic-PX, Dental Cone Beam CT-DCBCT, PET-PET, PET/CT-PETCT, SPECT-SPECT, SPECT/CT-SPECTCT, Nuclear Medicine-NM, Mobile X-ray-DXMOB, Conventional with fluoro-DXXA
        machines=["U110", "S08", "L3"]#, "U105", "U106"]        # Umeå, Skellefteå, Lycksele + IR
    )
    
    #rembox.filter_options.set_exclusive_tags() om jag vill ange filter där man bortser från ett visst kriterie

    rembox.filter_options.patient_age_interval_include_nulls = True
    
    rembox.filter_options.study_time_interval_start_date = "2024-01-01T00:00:00Z"
    rembox.filter_options.study_time_interval_end_date = "2024-12-31T00:00:00Z"
    
    
    rembox.add_columns(
        columns=[
            valid_study_columns.AccessionNumber,
            valid_study_columns.AcquisitionDoseAreaProductTotal,
            valid_study_columns.AcquisitionDoseRPTotal,
            valid_study_columns.AcquisitionPlane,
            #valid_study_columns.CalibrationDate,
            #valid_study_columns.CalibrationFactor,
            #valid_study_columns.CalibrationProtocol,
            #valid_study_columns.CalibrationResponsibleParty,
            #valid_study_columns.CalibrationUncertainty,
            valid_study_columns.City,
            valid_study_columns.ConvFluoroClassifier,
            valid_study_columns.DoseAreaProductTotal,
            #valid_study_columns.DoseMeasurementDevice,
            valid_study_columns.DoseRPTotal,
            valid_study_columns.FluoroDoseAreaProductTotal,
            valid_study_columns.FluoroDoseRPTotal,
            #valid_study_columns.HasIntent,
            #valid_study_columns.HalfValueLayer,
            valid_study_columns.Hospital,
            valid_study_columns.Id,
            valid_study_columns.Machine,
            #valid_study_columns.MeanBodyThickness,
            #valid_study_columns.MaximumBodyThickness,
            #valid_study_columns.MinimumBodyThickness,
            valid_study_columns.PatientAge,
            valid_study_columns.PatientAgeUnit,
            valid_study_columns.PatientDbId,
            valid_study_columns.PatientId,
            #valid_study_columns.PatientModel,
            #valid_study_columns.PatientsBodyMassIndex,
            #valid_study_columns.PatientsName,
            valid_study_columns.PatientsSex,
            valid_study_columns.PatientsSize,
            valid_study_columns.PatientsSizeDate,
            valid_study_columns.PatientsSizeSource,
            valid_study_columns.PatientsWeight,
            valid_study_columns.PatientsWeightDate,
            valid_study_columns.PatientsWeightSource,
            #valid_study_columns.PerformingPhysicianIdentificationSequence,
            valid_study_columns.PerformingPhysicianName,
            valid_study_columns.PregnancyStatus,
            valid_study_columns.ProcedureCode,
            valid_study_columns.ProcedureCodeMeaning,
            valid_study_columns.ProcedureReported,
            valid_study_columns.ProtocolCode,
            valid_study_columns.ProtocolCodeMeaning,
            valid_study_columns.PSD,
            #valid_study_columns.ReferenceAuthority,
            #valid_study_columns.ReferencedSopInstanceUid,
            #valid_study_columns.ReferencePointDefinition,
            #valid_study_columns.ReferencePointDefinitionCode,
            #valid_study_columns.ReferringPhysicianIdentificationSequence,
            #valid_study_columns.ReferringPhysiciansName,
            valid_study_columns.RequestedProcedureCode,
            valid_study_columns.RequestedProcedureCodeMeaning,
            valid_study_columns.ReferencePointDefinition,
            valid_study_columns.ScopeOfAccumulation,
            valid_study_columns.SoftwareVersions,
            valid_study_columns.SourceOfDoseInformation,
            valid_study_columns.StartOfXrayIrradiation,
            valid_study_columns.StudyDateTime,
            valid_study_columns.StudyDescription,
            valid_study_columns.StudyId,
            valid_study_columns.StudyInstanceUID,
            valid_study_columns.TotalAcquisitionTime,
            valid_study_columns.TotalFluoroTime,
            valid_study_columns.TotalNumberOfIrradiationEvents,
            valid_study_columns.TotalNumberOfRadiographicFrames,
            valid_series_columns.AcquisitionPlaneSeries,
            valid_series_columns.AcquisitionProtocol,
            #valid_series_columns.AcquisitionType,
            #valid_series_columns.ApplicationName, #--------------------------------------Här finns protokollnamn för Azurion
            #valid_series_columns.AnatomicalStructure,
            #valid_series_columns.AnodeTargetMaterial,
            #valid_series_columns.AverageXrayTubeCurrent,
            valid_series_columns.CollimatedFieldArea,
            valid_series_columns.CollimatedFieldHeight,
            valid_series_columns.CollimatedFieldWidth,
            #valid_series_columns.ColumnAngulation,
            #valid_series_columns.CrdrMechanicalConfiguration,
            valid_series_columns.DateTimeStarted,
            #valid_series_columns.DerivedEffectiveDiameter,
            #valid_series_columns.DeviationIndex,
            valid_series_columns.DistanceSourceToDetector,
            valid_series_columns.DistanceSourceToIsocenter,
            valid_series_columns.DistanceSourceToReferencePoint,
            #valid_series_columns.DistanceSourceToTablePlane,
            valid_series_columns.DoseAreaProduct,
            valid_series_columns.DoseRP,
            #valid_series_columns.EffectiveDose,
            #valid_series_columns.EffectiveDoseConversionFactor,
            #valid_series_columns.EntranceExposureAtRP,
            #valid_series_columns.ExposedRange,
            valid_series_columns.Exposure,
            #valid_series_columns.ExposureIndex,
            valid_series_columns.ExposureTime,
            #valid_series_columns.ExposureTimePerRotation,
            valid_series_columns.XrayFilterAluminumEquivalent,
            valid_series_columns.XrayFilterType,
            valid_series_columns.XrayFilterMaterial,
            valid_series_columns.XrayFilterThicknessMaximum,
            valid_series_columns.XrayFilterThicknessMinimum,
            valid_series_columns.FluoroMode,
            #valid_series_columns.FluoroFlavour, # -------------------------------------------- Här finns pulsrat för Azurion
            #valid_series_columns.FrameOfReferenceUID,
            #valid_series_columns.IdentificationOfTheXraySource,
            #valid_series_columns.ImageView,
            #valid_series_columns.ImageViewModifier,
            #valid_series_columns.XrayGrid,
            #valid_series_columns.XrayGridPitch,
            #valid_series_columns.XrayGridAspectRatio,
            #valid_series_columns.XrayGridFocalDistance,
            valid_series_columns.IrradiationDuration,
            #valid_series_columns.IrradiationEventLabel,
            valid_series_columns.IrradiationEventType,
            valid_series_columns.IrradiationEventUID,
            valid_series_columns.kVp,
            valid_series_columns.XrayTubeCurrent,
            #valid_series_columns.XrayModulationType,
            #valid_series_columns.LabelType,
            #valid_series_columns.Laterality,
            #valid_series_columns.MaximumXrayTubeCurrent,
            #valid_series_columns.MeasurementMethodDose,
            #valid_series_columns.NominalCollimationWidth,
            #valid_series_columns.NominalTotalCollimationWidth,
            valid_series_columns.NumberOfPulses,
            #valid_series_columns.NumberOfXraySources,
            valid_series_columns.PatientEquivalentThickness,
            valid_series_columns.PatientOrientation,
            valid_series_columns.PatientOrientationModifier,
            valid_series_columns.PatientTableRelationship,
            valid_series_columns.PositionerPrimaryAngle,
            #valid_series_columns.PositionerPrimaryEndAngle,
            valid_series_columns.PositionerSecondaryAngle,
            #valid_series_columns.PositionerSecondaryEndAngle,
            #valid_series_columns.ProcedureContext,
            #valid_series_columns.ProjectionEponymousName,
            valid_series_columns.PulseRate,
            valid_series_columns.PulseWidth,
            #valid_series_columns.ReconstructionAlgortihm,
            #valid_series_columns.ReferencePointDefinitionText,
            valid_series_columns.SpotSize,
            valid_series_columns.TableCradleTiltAngle,
            valid_series_columns.TableHeadTiltAngle,
            valid_series_columns.TableHeightEndPosition,
            valid_series_columns.TableHeightPosition,
            valid_series_columns.TableHorizontalRotationAngle,
            valid_series_columns.TableLateralEndPosition,
            valid_series_columns.TableLateralPosition,
            valid_series_columns.TableLongitudinalEndPosition,
            valid_series_columns.TableLongitudinalPosition,
            #valid_series_columns.TargetExposureIndex,
            valid_series_columns.TargetRegion,
            #valid_series_columns.WaterEquivalentDiameter,
            #valid_series_columns.WedMeasurementMethod,
            #valid_series_columns.XrayFilterAluminumEquivalent,
            #valid_series_columns.XrayFilterMaterial,
            #valid_series_columns.XrayFilterThicknessMaximum,
            #valid_series_columns.XrayFilterThicknessMinimum,
            #valid_series_columns.XrayFilterType,
            #valid_series_columns.XrayGrid,
            #valid_series_columns.XrayGridAspectRatio,
            #valid_series_columns.XrayGridFocalDistance,
            #valid_series_columns.XrayGridPitch,
            #valid_series_columns.XrayModulationType,
            #valid_series_columns.XrayTubeCurrent
        ]
    )

    return rembox.run_query()

In [ ]:
#Hämta data från REMbox
study_data, series_data = get_data_from_fluoro(rembox=rembox)

In [ ]:
#Export av data till csv

#study_data.to_csv("C:/Users/chgr09/GIT/rvbrtg/Data/output_data/XA_study_2023.csv")
#series_data.to_csv("C:/Users/chgr09/GIT/rvbrtg/Data/output_data/XA_series_2023.csv")

## Kontroller av data och hantering av dataframes

In [ ]:
study = study_data.copy() #skapa kopia av dataframe på study-nivå för att kunna behålla orginalet
series = series_data.copy() #skapa kopia av dataframe på serie-nivå för att kunna behålla orginalet

# TODO: Ta bort testpatienter
#Ta bort undersökningar från testpatient U110 som har PatientId = ??
#Ta bort undersökningar från testpatient S08 som har PatientId = ??
#Ta bort undersökningar från testpatient L3 som har PatientId = ??

In [ ]:
#study.head()

In [ ]:
fig = px.scatter(study, x="studyDateTime", y="doseAreaProductTotal", color="machine")

fig.show()

In [ ]:
#Räkna antal ingrepp och antal unika patienter
exams = study["patientDbId"].count()
patients = study["patientDbId"].nunique() #Count number of distinct elements in specified axis. Can ignore NaN values

print(exams, "undersökningar/ingrepp fördelat på", patients, "patienter")

In [ ]:
# TODO: Kolla om det är första eller sista instansen av samma Accnr som ska sparas. Vad händer när data kommer från bilder?

print(len(study), "antal undersökningar")

unique_studies = study.drop_duplicates(subset="accessionNumber", keep="first")

no_acc_numbers = study["accessionNumber"].count()
no_unique_acc_numbers = unique_studies["accessionNumber"].count()

print(no_acc_numbers, "AccessionNumbers i ursprungling dataframe med", no_unique_acc_numbers, "unika AccessionNumbers")

In [ ]:
print(len(series), "antal irradiation events")

unique_series = series.drop_duplicates(subset="irradiationEventUID", keep="first")

no_irradiations = series["irradiationEventUID"].count()
no_unique_irradiations = unique_series["irradiationEventUID"].nunique()

print(no_irradiations, "irradiation events i ursprungling dataframe med", no_unique_irradiations, "unika irradiationEventUID")

## Lägg till operatörsnamn

In [ ]:
# Översättningstabell från pseudo-operatörer till operatörer
names_data_path = "C:/Projekt/GIT/rvbrtg/Data/input_data/operators_2025.xlsx" 

names = pd.read_excel(names_data_path)

#Ändra kolumnnamn för att kunna göra en join
names.columns = ["performingPhysicianName", "OperatorName"]
#names.head()

#Joina dataframes för att få in operatörsnamn
study_names = unique_studies.merge(names, on = ["performingPhysicianName"], how = "left")

#Print för att kolla så att det funkade
#print(study_names.OperatorName)


In [ ]:
#Granska om namn saknas
study_names.to_csv("C:/Projekt/GIT/rvbrtg/Data/output_data/Operators_missing_GML_2024.csv")

## Analysera Data

In [ ]:
#Normalisera DAP till genomlysningstid och till som kolumn i dataframe

study_names['normalisedFluoroDap'] = study_names['fluoroDoseAreaProductTotal'] / study_names['totalFluoroTime']

In [ ]:
# Tabell över ackumulerad DAP, total DAP och antal ingrepp per Kod.

procedure_dap = (study_names.groupby(["procedureCode", "procedureCodeMeaning", "machine"])
                    .agg({'doseAreaProductTotal': ['sum', 'median'], 'procedureCode': 'count'}
                         ).reset_index().round(3))
procedure_dap.head()

In [ ]:
fig = px.box(study_names, x='procedureCodeMeaning', y='doseAreaProductTotal', color='machine')
#fig.update_layout(height=1000)
fig.show()


## Skellefteå

In [ ]:
study_skellefteå = study_names[study_names['machine'] == 'S08']
#study_skellefteå.head()

In [ ]:
median_KAP_studytype_sk = (study_skellefteå.groupby(
    ["procedureCode", "procedureCodeMeaning", "OperatorName"])
                           .agg({'doseAreaProductTotal': ['min', 'median', 'max'], 'normalisedFluoroDap': 'median', 'procedureCode': 'count'}
                                 ).reset_index().round(3)) 
median_KAP_studytype_sk

In [ ]:
fig = px.box(study_skellefteå, x='procedureCodeMeaning', y='doseAreaProductTotal')#, color='OperatorName')  
#fig.update_layout(height=800)  
fig.show()

In [ ]:
removals = study_skellefteå['procedureCode'].value_counts().reset_index()
removals = removals[removals['count'] > 10] # Ändra tröskel för hur många undersökningar som måste finnas med samma Kod i dataframe
removals

filtered_df = study_skellefteå[study_skellefteå['procedureCode'].isin(removals['procedureCode'])]
#filtered_df.head()

fig = px.box(filtered_df, x='procedureCodeMeaning', y='doseAreaProductTotal', points='all')
#fig.update_layout(width=1000, height=600)
fig.show()

In [ ]:
# Plocka ut de Koder som används i Skellefteå och jämför dem med Umeå och Lycksele

filtered_df = study_names[study_names['procedureCode'].isin(study_skellefteå['procedureCode'])]

fig = px.box(filtered_df, x='procedureCodeMeaning', y='doseAreaProductTotal', color='machine')
#fig.update_layout(height=1000)
fig.show()

## Umeå

In [ ]:
study_umeå = study_names[study_names['machine'] == 'U110']
#study_umeå.head()

In [ ]:
median_KAP_studytype_uå = (study_umeå.groupby(
    ["procedureCode", "procedureCodeMeaning", "OperatorName"]) 
                           .agg({'doseAreaProductTotal': ['min', 'median', 'max'], 'doseRPTotal': 'median', 'normalisedFluoroDap': 'median', 'procedureCode': 'count'}
                                 ).reset_index().round(3))
median_KAP_studytype_uå

In [ ]:
fig = px.box(study_umeå, x='procedureCodeMeaning', y='doseAreaProductTotal')#, 
#fig.update_layout(width=1000, height=600)
fig.show()

In [ ]:
removals = study_umeå['procedureCode'].value_counts().reset_index()
removals = removals[removals['count'] > 10] # Ändra tröskel för hur många undersökningar som måste finnas med samma Kod i dataframe
removals

filtered_df = study_umeå[study_umeå['procedureCode'].isin(removals['procedureCode'])]
#filtered_df.head()

fig = px.box(filtered_df, x='procedureCodeMeaning', y='doseAreaProductTotal', points='all')
#fig.update_layout(width=1000, height=600)
fig.show()

In [ ]:
# Barnundersökningar

fig = px.box(study_umeå[study_umeå['patientAge'] < 18], x='procedureCodeMeaning', y='doseAreaProductTotal', points='all')
#fig.update_layout(width=1000, height=600)
fig.show()

## Lycksele

In [ ]:
study_lycksele = study_names[study_names['machine'] == 'L3']

In [ ]:
median_KAP_studytype_ly = (study_lycksele.groupby(
    ["procedureCode", "procedureCodeMeaning", "OperatorName"])
                           .agg({'doseAreaProductTotal': ['min', 'median', 'max'], 'normalisedFluoroDap': 'median', 'procedureCode': 'count'}
                                 ).reset_index().round(3))
median_KAP_studytype_ly

In [ ]:
fig = px.box(study_lycksele, x='procedureCodeMeaning', y='doseAreaProductTotal')
#fig.update_layout(width=1000, height=600)
fig.show()

In [ ]:
removals = study_lycksele['procedureCode'].value_counts().reset_index()
removals = removals[removals['count'] > 1] # Ändra tröskel för hur många undersökningar som måste finnas med samma Kod i dataframe
removals

filtered_df = study_lycksele[study_lycksele['procedureCode'].isin(removals['procedureCode'])]
#filtered_df.head()

fig = px.box(filtered_df, x='procedureCodeMeaning', y='doseAreaProductTotal', points='all')
#fig.update_layout(width=1000, height=600)
fig.show()

In [ ]:
# Plocka ut de Koder som används i Lycksele och jämför dem med Umeå och Skellefteå

filtered_df = study_names[study_names['procedureCode'].isin(study_lycksele['procedureCode'])]

fig = px.box(filtered_df, x='procedureCodeMeaning', y='doseAreaProductTotal', color='machine')
#fig.update_layout(height=1000)
fig.show()


## Individuella undersökningar

In [ ]:
fig = px.scatter(study[study.protocolCode == "59100"], x="studyDateTime", y="doseRPTotal", color="machine")
fig.update_layout(title_text='Nefrobyte')
fig.show()

In [ ]:
fig = px.scatter(study[study.protocolCode == "49600"], x="patientAge", y="doseAreaProductTotal", color="machine")
fig.update_layout(title_text='Invagination')
fig.show()

49600

In [ ]:
median_KAP_nefro = (study_names[study_names['procedureCode'] == '59100'].groupby(
    ["procedureCodeMeaning", "OperatorName", "machine"])
                           .agg({'doseAreaProductTotal': ['min', 'median', 'max'], 'normalisedFluoroDap': 'median', 'procedureCode': 'count'}
                                 ).reset_index().round(2))
median_KAP_nefro

In [ ]:
fig = px.scatter(study_names[(study_names.protocolCode == "59100") & (study_names.machine == 'S08')], x="totalFluoroTime", y="fluoroDoseAreaProductTotal", color="OperatorName", hover_name="OperatorName", hover_data=["fluoroDoseAreaProductTotal", "totalFluoroTime", "accessionNumber"])
fig.update_layout(title_text='Byte av nefrostomikateter')
fig.show()
#fig.write_html("C:/Projekt/GIT/rvbrtg/Data/output_data/html_plot_accnr.html")

In [ ]:
nefro = study_names[study_names['procedureCode'] == '59100']
removals = nefro['OperatorName'].value_counts().reset_index()
removals = removals[removals['count'] > 8] # Ändra tröskel för hur många undersökningar som måste finnas med samma Kod i dataframe
#removals

filtered_df = nefro[nefro['OperatorName'].isin(removals['OperatorName'])]
#filtered_df.head()

fig = px.box(filtered_df, x='OperatorName', y='doseAreaProductTotal', points='all')
#fig.update_layout(width=1000, height=600)
fig.show()

## Analysera om det finns detaljerade skillnader som val av pulshastighet, dos-mode, antal exponeringar, etc.

In [ ]:
# Plocka ut medelvärde för kollimerad area från varje undersökning och plotta med detta som färg på datapunkter

def get_coll_avg(row) -> float:
    return series.loc[series.studyId == row.id].collimatedFieldArea.mean()

study["collAvg"] = [
    get_coll_avg(row)
    for row in study.itertuples()
]

#study.head()
fig = px.scatter(study[(study.protocolCode == "59100") & (study.machine == "S08")], x="totalFluoroTime", y="fluoroDoseAreaProductTotal", color="collAvg")
fig.update_layout(title_text='Byte av nefrostomikateter')
fig.show()

In [ ]:
# Top 20 i Skellefteå, Umeå och Lycksele
top_nefro = study_names[study_names.procedureCode == "59100"].nlargest(20, 'fluoroDoseAreaProductTotal')
top_20_skel = study_names[(study_names.procedureCode == "59100") & (study_names.machine == "S08")].nlargest(20, 'fluoroDoseAreaProductTotal')
top_20_ume = study_names[(study_names.procedureCode == "59100") & (study_names.machine == "U110")].nlargest(20, 'fluoroDoseAreaProductTotal')
top_20_lyck = study_names[(study_names.procedureCode == "59100") & (study_names.machine == "L3")].nlargest(20, 'fluoroDoseAreaProductTotal')
top_nefro.head()

In [ ]:
#Joina dataframes för att få study och series i samma dataframe
top_stud_ser_gml = top_nefro.merge(series, on=["studyInstanceUID"], how="left")

top_stud_ser_gml = top_stud_ser_gml.drop('accessionNumber_y', axis=1)
top_stud_ser_gml = top_stud_ser_gml.rename(columns={'accessionNumber_x': 'accessionNumber'})
top_stud_ser_gml = top_stud_ser_gml.drop('studyId_y', axis=1)
top_stud_ser_gml = top_stud_ser_gml.rename(columns={'studyId_x': 'studyId'})

#Print för att kolla så att det funkade
#print(top_stud_ser_gml.columns)

In [ ]:
fig = px.scatter(top_stud_ser_gml[top_stud_ser_gml.accessionNumber == "SERSKE0008140047"], x="dateTimeStarted", y="doseAreaProduct", color="pulseRate")
fig.update_layout(title_text='Byte av nefrostomikateter')
fig.show()

In [ ]:
fig = px.scatter(top_stud_ser_gml[top_stud_ser_gml.accessionNumber == "SERSKE0008140047"], x="dateTimeStarted", y="doseAreaProduct", color="collimatedFieldArea")
fig.update_layout(title_text='Byte av nefrostomikateter')
fig.show()

In [ ]:
fig = px.scatter(top_stud_ser_gml[top_stud_ser_gml.accessionNumber == "SERSKE0008140047"], x="dateTimeStarted", y="doseAreaProduct", color="acquisitionProtocol")
fig.update_layout(title_text='Byte av nefrostomikateter')
fig.show()

In [ ]:
fig = px.scatter(top_stud_ser_gml[top_stud_ser_gml.accessionNumber == "SERSKE0008140047"], x="dateTimeStarted", y="doseAreaProduct", color="numberOfPulses")
fig.update_layout(title_text='Byte av nefrostomikateter')
fig.show()

In [ ]:
median_KAP_nefro_ser = (top_stud_ser_gml[top_stud_ser_gml['procedureCode'] == '59100'].groupby(
    ["accessionNumber", "OperatorName", "machine", "doseAreaProductTotal", "acquisitionProtocol", "pulseRate"])
                           .agg({'doseAreaProduct': 'sum', 'procedureCode': 'count'}
                                 ).reset_index().round(2))

final_df = median_KAP_nefro_ser.sort_values(by=['doseAreaProductTotal'], ascending=False)
final_df

## Dump till csv

In [ ]:
#Testa dumpa ut en undersökning för att granska vidare

test = series_data[series_data.accessionNumber == "SERUME0008272496"]
test.to_csv("C:/Projekt/GIT/rvbrtg/Data/output_data/DUMP_XXX.csv")